#### ANÁLISIS DE SENTIMIENTO 
Se realiza una clasificación para los diferentes audios recolectados en llamadas al servicio al cliente (20) en específico los últimos 15 segundos, para  clasificar la atención prestada y rendimiento del servicio

In [245]:
import speech_recognition as sr
import pandas as pd

In [246]:
import os

#ABRIMOS LA CARPETA DE LOS AUDIOS
folder_audios = os.listdir("AUDIOS")

In [247]:
def transcribe_audio(audio):
    recognizer = sr.Recognizer()
    audio_new = sr.AudioFile(audio)
    with audio_new as source:
        audio_senti = recognizer.record(source)
    return recognizer.recognize_google(audio_data=audio_senti, language="es-CO")

In [248]:
def create_text_list(folder):
    text_list = []
    for file in folder:
         if file.endswith(".wav"):
            text = transcribe_audio(file)
            text_list.append(text)
    return text_list

In [249]:
text_audio_sent = create_text_list(folder_audios)

In [250]:
#CREAMOS EL DATAFRAME PARA TODAS LAS RESEÑAS RECOGIDAS
df_sentiment_call = pd.DataFrame({ "Reseña": text_audio_sent})

In [251]:
from textblob import TextBlob

In [252]:
df_sentiment_call['Reseña_Ingles'] = df_sentiment_call['Reseña'].apply(lambda x: str(TextBlob(x).translate(from_lang='es',to='en')))

In [253]:
df_sentiment_call['Polaridad'] = df_sentiment_call['Reseña_Ingles'].apply(lambda x: TextBlob(x).sentiment.polarity)
df_sentiment_call['Subjetividad'] = df_sentiment_call['Reseña_Ingles'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [254]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

df_sentiment_call['Position'] = df_sentiment_call['Reseña_Ingles'].apply(lambda x: analyzer.polarity_scores(x))

In [256]:
df_sentiment_call['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df_sentiment_call['Reseña_Ingles']]

In [257]:
df_sentiment_call['Clasificacion_vader'] = df_sentiment_call['compound'].apply(lambda x: 'Positivo' if x > 0.05 else ('Negativo' if x < -0.05 else 'Neutral'))
df_sentiment_call['Clasificacion_TextBlod'] = df_sentiment_call['Polaridad'].apply(lambda x: 'Positivo' if x > 0.0 else ('Negativo' if x < 0.0 else 'Neutral'))

In [258]:
#CLASIFICACION 6 PRIMERAS LLAMADAS
df_sentiment_call.head(6)

,Reseña,Reseña_Ingles,Polaridad,Subjetividad,Position,compound,Clasificacion_vader,Clasificacion_TextBlod
0,Bueno pues la verdad ese servicio es muy muy m...,"Well, the truth is, that service is very very ...",-0.673333,0.611111,"{'neg': 0.223, 'neu': 0.657, 'pos': 0.12, 'com...",-0.6949,Negativo,Negativo
1,el proceso de instalación con muy bueno recibi...,the installation process with very good receiv...,0.840000,0.720000,"{'neg': 0.0, 'neu': 0.591, 'pos': 0.409, 'comp...",0.8945,Positivo,Positivo
2,este servicio me ha parecido regular Ya que en...,This service has seemed to me regular Since at...,-0.025000,0.359103,"{'neg': 0.133, 'neu': 0.812, 'pos': 0.055, 'co...",-0.5859,Negativo,Negativo
3,no tenía ningún inconveniente con el servicio ...,I had no problem with the one-time service in ...,0.000000,0.000000,"{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp...",0.3089,Positivo,Neutral
4,desagradable tenido fallos en la red durante t...,unpleasant had network failures throughout the...,-0.650000,0.950000,"{'neg': 0.357, 'neu': 0.643, 'pos': 0.0, 'comp...",-0.7269,Negativo,Negativo
5,muy muy malo espero que en un futuro mejor en ...,very very bad I hope that in a better future i...,-0.136667,0.497222,"{'neg': 0.176, 'neu': 0.562, 'pos': 0.262, 'co...",0.2482,Positivo,Negativo


La clasificacion se muestra para las dos librerias posibles para el estudio y su funcionamiento durante el analisis, se puede evidenciar que ambas funcionan para ciertas llamadas con clasificacion desigual y deben ser corregidas, los resultados pueden ser objeto de un posterior algoritmo de clasificacion o segmentacion de clientes. 